##Instituto Tecnológico y de Estudios Superiores de Monterrey

### Proyecto Base de Datos de Big Data Equipo 38

- Salvador Hernández Medrano A01796607

- Emmanuel Merida Toledo A01795858

- Francisco Gómez Rubio A00710437

- Josué Daniel Lescano López A01367234

### Análisis de grandes volúmenes de datos

4 de Mayo de 2025

# Cryptocurrency 234 Coins Altcoins Prices

El dataset contiene precios históricos de 234 criptomonedas (incluyendo Bitcoin y altcoins), con los siguientes campos para cada transacción:
•	Open (precio de apertura)

•	High (precio máximo)

•	Low (precio mínimo)

•	Close (precio de cierre)

•	Volume (volumen operado)


Estos datos están disponibles en diferentes marcos de tiempo: W1 (Semanal), D1 (Diaria), H4 (4 horas), H1 (1 hora), M30 (30 minutos), M15 (15 minutos) y M5 (cinco minutos).


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,mean, avg, min, max, count, lit, round, concat_ws
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#lectura del archivo
spark = SparkSession.builder.master("local[*]").getOrCreate()
df = spark.read.csv("/content/drive/MyDrive/BIG DATA/Proyecto 1/dataset.csv", header=True, inferSchema=True)
df.show()

+-------------------+--------+--------+--------+--------+---------+------+---------+
|           datetime|    open|    high|     low|   close|   volume|  coin|frequency|
+-------------------+--------+--------+--------+--------+---------+------+---------+
|2017-07-14 05:00:00|    0.08|0.091033|    0.08|0.090993| 1942.057|ETHBTC|       D1|
|2017-07-15 05:00:00|0.090993|0.093699|0.087127|0.087635| 4013.066|ETHBTC|       D1|
|2017-07-16 05:00:00|0.087508|0.087635|0.075591|0.082241| 8904.158|ETHBTC|       D1|
|2017-07-17 05:00:00|0.082368|0.088394|0.081699|0.087537| 6650.933|ETHBTC|       D1|
|2017-07-18 05:00:00|0.087831|0.109068|0.084777|0.107732| 7245.741|ETHBTC|       D1|
|2017-07-19 05:00:00|0.107732|0.108732| 0.08429|0.086853| 7384.976|ETHBTC|       D1|
|2017-07-20 05:00:00|0.087144|0.095252|0.074982|0.080509|  6395.16|ETHBTC|       D1|
|2017-07-21 05:00:00|0.080383|0.087511|0.079635|0.081581| 7434.389|ETHBTC|       D1|
|2017-07-22 05:00:00|0.081586|0.084733|0.079856|0.082474| 7979.89

In [5]:
#1. Caracterización de la población.

#principales caracteriscticas de variables númericas
numerical_stats = df.select(
    mean("open").alias("mean_open"), avg("open").alias("avg_open"), min("open").alias("min_open"), max("open").alias("max_open"),
    mean("high").alias("mean_high"), avg("high").alias("avg_high"), min("high").alias("min_high"), max("high").alias("max_high"),
    mean("low").alias("mean_low"), avg("low").alias("avg_low"), min("low").alias("min_low"), max("low").alias("max_low"),
    mean("close").alias("mean_close"), avg("close").alias("avg_close"), min("close").alias("min_close"), max("close").alias("max_close"),
    mean("volume").alias("mean_volume"), avg("volume").alias("avg_volume"), min("volume").alias("min_volume"), max("volume").alias("max_volume")
)

numerical_stats.show(truncate=False,vertical=True)


-RECORD 0-------------------------
 mean_open   | 315.30808366873765 
 avg_open    | 315.30808366873765 
 min_open    | 6.56E-5            
 max_open    | 90877.82           
 mean_high   | 316.24911175329464 
 avg_high    | 316.24911175329464 
 min_high    | 6.63E-5            
 max_high    | 95000.0            
 mean_low    | 314.33606158631443 
 avg_low     | 314.33606158631443 
 min_low     | 6.55E-5            
 max_low     | 90211.0            
 mean_close  | 315.3110150076161  
 avg_close   | 315.3110150076161  
 min_close   | 6.56E-5            
 max_close   | 90912.32           
 mean_volume | 3781457.4590252754 
 avg_volume  | 3781457.4590252754 
 min_volume  | 0.0                
 max_volume  | 3.326897912677E12  



In [8]:
total_count = df.count()

# Frecuencia absoluta top 5 de 'coin'
df.groupBy("coin") \
  .count() \
  .withColumn("percentage", round((col("count") / total_count) * 100, 2)) \
  .orderBy("count", ascending=False) \
  .limit(5) \
  .show(truncate=False)

+-------+-------+----------+
|coin   |count  |percentage|
+-------+-------+----------+
|ETHBTC |1125355|0.9       |
|BTCUSDT|1109604|0.89      |
|ETHUSDT|1109604|0.89      |
|BNBUSDT|1072225|0.86      |
|NEOUSDT|1065744|0.85      |
+-------+-------+----------+



In [7]:
# Frecuencia absoluta top 5 de 'frequency'
df.groupBy("frequency").count().orderBy("count", ascending=False).limit(5).show(truncate=False)

+---------+--------+
|frequency|count   |
+---------+--------+
|M5       |77826095|
|M15      |25942097|
|M30      |12971381|
|H1       |6486216 |
|H4       |1622504 |
+---------+--------+



In [9]:
# las 5 monedas principales

coin_counts = df.groupBy("coin").count().orderBy("count", ascending=False).limit(5).toPandas()
coin_counts.columns = ["coin", "coin_count"]
freq_counts = df.groupBy("frequency").count().orderBy("count", ascending=False).limit(5).toPandas()
freq_counts.columns = ["frequency", "freq_count"]

combined = pd.concat([coin_counts, freq_counts], axis=1)
print(combined)

      coin  coin_count frequency  freq_count
0   ETHBTC     1125355        M5    77826095
1  BTCUSDT     1109604       M15    25942097
2  ETHUSDT     1109604       M30    12971381
3  BNBUSDT     1072225        H1     6486216
4  NEOUSDT     1065744        H4     1622504


In [7]:
#Particonamiento
# Total de registros
combinaciones = [
    ("ETHBTC", "M5", 0.06),  # Combinación A -> a, B -> c
    ("BTCUSDT", "M15", 0.24),  # Combinación A -> a, B -> d
    ("ETHUSDT", "M30", 0.14),  # Combinación A -> b, B -> c
    ("BNBUSDT", "H1", 0.56)   # Combinación A -> b, B -> d
]

# **7. Recuperar submuestras según las reglas de particionamiento**

# Iterar sobre las combinaciones y filtrar los registros de acuerdo con las reglas definidas
for comb in combinaciones:
    a_value = comb[0]
    b_value = comb[1]
    probabilidad = comb[2]

    # Filtrar el DataFrame según la combinación
    particion = df.filter((col("coin") == a_value) & (col("frequency") == b_value))

    # Mostrar la partición filtrada y su probabilidad
    print(f"Combinación: Coin = {a_value}, Frequency = {b_value}, Probabilidad = {probabilidad}")
    particion.show()

    # Guardar la partición en un archivo CSV (opcional)
    #particion.write.csv(f"output_{a_value}_{b_value}", header=True)

Combinación: Coin = ETHBTC, Frequency = M5, Probabilidad = 0.06
+-------------------+--------+--------+--------+--------+------+------+---------+
|           datetime|    open|    high|     low|   close|volume|  coin|frequency|
+-------------------+--------+--------+--------+--------+------+------+---------+
|2017-07-14 09:00:00|    0.08|    0.08|    0.08|    0.08| 0.726|ETHBTC|       M5|
|2017-07-14 09:05:00|    0.08|0.080001|    0.08|0.080001| 3.347|ETHBTC|       M5|
|2017-07-14 09:10:00|0.080001|  0.0864|0.080001|  0.0864| 4.679|ETHBTC|       M5|
|2017-07-14 09:15:00|0.085289| 0.08562|0.085128|0.085128|53.431|ETHBTC|       M5|
|2017-07-14 09:20:00|0.085274|   0.086|0.085274|   0.086| 5.576|ETHBTC|       M5|
|2017-07-14 09:25:00| 0.08525|0.085811| 0.08525|0.085811| 2.035|ETHBTC|       M5|
|2017-07-14 09:30:00|0.085811|   0.086|0.085811|   0.086|17.943|ETHBTC|       M5|
|2017-07-14 09:35:00| 0.08618| 0.08618| 0.08618| 0.08618|18.237|ETHBTC|       M5|
|2017-07-14 09:40:00| 0.08618| 0.0